#  Training Model- Solution C - BERT

In [ ]:
# Training Model - Solution C - BERT

!cat /proc/cpuinfo | grep "model name" | uniq
!nvidia-smi
# Please install these libraries below
# !pip install torch
# !pip install transformers
# !pip install tqdm
# !pip install pandas
# !pip install numpy
# !pip install scikit-learn

# Use the A100 GPU to run this code

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import re
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import f1_score, accuracy_score

# File path of dataset
train_file_path = '/content/drive/My Drive/nlu-lab/train.csv'
validation_file_path = '/content/drive/My Drive/nlu-lab/dev.csv'

# Load training and testing data
train_data = pd.read_csv(train_file_path)
val_data = pd.read_csv(validation_file_path)

# Check the data loading
print(train_data.head(2))
print(val_data.head(2))

model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
Thu Apr 10 13:14:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             49W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+---

# Text Preprocessing

In [ ]:
# text preprocessing
def clean_text(text):
  text = re.sub(r"can't\b", "cannot", text)  # Replace can't with cannot
  text = re.sub(r"won't\b", "will not", text)  # Replace won't with will not
  text = re.sub(r"n't\b", " not", text)  # Replace n't with not
  text = re.sub(r"'re\b", " are", text)  # Replace 're with are
  text = re.sub(r"'m\b", " am", text)  # Replace 'm with am
  text = re.sub(r"'ve\b", " have", text)  # Replace 've with have
  text = re.sub(r"'ll\b", " will", text)  # Replace 'll with will
  text = re.sub(r"'d\b", " would", text)  # Replace 'd with would
  # Expand only pronoun + 's
  text = re.sub(r"\b(he|she|it|that|what|who|there|where|why|when)'s\b", r"\1 is", text, flags=re.IGNORECASE)
  # url
  text = re.sub(r'http\S+|www\S+|https\S+', '[URL]', text)
  # blank space
  text = re.sub(r"\s+", " ", text).strip()
  return text

train_data['combined_text'] = (train_data['Claim'] + " " + train_data['Evidence']).apply(clean_text)
val_data['combined_text'] = (val_data['Claim'] + " " + val_data['Evidence']).apply(clean_text)

# Check the cleaned data
print(train_data[['combined_text']].head())
print(val_data[['combined_text']].head())

                                       combined_text
0  We should introduce school vouchers Among the ...
1  We should legalize insider trading The U.S. Se...
2  We should subsidize investigative journalism T...
3  We should further exploit nuclear power a 2001...
4  We should ban whaling The US and several other...
                                       combined_text
0  Democracy should be done away with. Amartya Se...
1  Polygamy should be made legal. The Supreme Cou...
2  Hunting should be banned In total it is estima...
3  Television should be given up. Barbera mention...
4  Abortions ought to be prohibited. According to...


# Training & Evaluation Function

In [ ]:
# Parameters
MAX_LEN = 128

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

#load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Dataset class
class EDDataset(Dataset):
  def __init__(self, texts, labels=None):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, index):
    text = self.texts[index]
    encoding = self.tokenizer(text, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='pt')
    item = {
        'input_ids': encoding['input_ids'].squeeze(0),
        'attention_mask': encoding['attention_mask'].squeeze(0)
    }
    if self.labels is not None:
      item['labels'] = torch.tensor(self.labels[index], dtype=torch.long)
    return item

# Function to train model, tqdm used for clear training process
from tqdm import tqdm

def train_model(model, data_loader, optimizer, criterion, device, num_epochs, show_progress=True):
  model.train()
  for epoch in tqdm(range(1, num_epochs + 1), desc="Epoch"):
    total_loss = 0
    loop = data_loader

    for batch in loop:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      optimizer.zero_grad()
      outputs = model(input_ids, attention_mask=attention_mask)
      loss = criterion(outputs.logits, labels)
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch}: Loss = {avg_loss:.4f}")

# Function to evaluate model
def evaluate_model(model, data_loader, device):
  model.eval()
  all_preds, all_labels = [], []
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask)
      preds = torch.argmax(outputs.logits, dim=1)

      all_preds.extend(preds.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())

  acc = accuracy_score(all_labels, all_preds)
  f1 = f1_score(all_labels, all_preds)
  return acc, f1


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Model Training

In [ ]:
from itertools import product
import os
import json
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertConfig, BertForSequenceClassification
from torch.optim import AdamW
import torch

# parameter gird search
param_grid = {
    'learning_rate': [1e-5, 2e-5],
    'batch_size': [8, 16, 32],
    'epochs': [3, 5],
    'dropout': [0.1, 0.3, 0.5],
}

# parameter combination list
param_combinations = list(product(
    param_grid['learning_rate'],
    param_grid['batch_size'],
    param_grid['epochs'],
    param_grid['dropout'],
))

best_f1 = 0
best_params = {}

model_dir = '/content/drive/My Drive/nlu-lab/bert_model'
os.makedirs(model_dir, exist_ok=True)

results = []

# training
for idx, (lr, bs, epochs, dropout) in enumerate(tqdm(param_combinations, desc="Grid Search Progress")):
  tqdm.write(f"\n[{idx+1}/{len(param_combinations)}] Training with lr={lr}, bs={bs}, epochs={epochs}, dropout={dropout}")

  train_dataset = EDDataset(train_data['combined_text'].tolist(), train_data['label'].tolist())
  val_dataset = EDDataset(val_data['combined_text'].tolist(), val_data['label'].tolist())
  train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=bs)

  # bert configuration
  config = BertConfig.from_pretrained(
      'bert-base-uncased',
      num_labels=2,
      hidden_dropout_prob=dropout,
      attention_probs_dropout_prob=dropout
  )
  model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config).to(device)

  optimizer = AdamW(model.parameters(), lr=lr)
  criterion = nn.CrossEntropyLoss()

  train_model(model, train_loader, optimizer, criterion, device, epochs, show_progress=True)

  acc, f1 = evaluate_model(model, val_loader, device)
  tqdm.write(f"Validation Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")

  results.append({
      'learning_rate': lr,
      'batch_size': bs,
      'epochs': epochs,
      'dropout': dropout,
      'acc': acc,
      'f1': f1
  })
  # Choose the best parameters according to F1 scores, then save the best model.
  if f1 > best_f1:
    best_f1 = f1
    best_params = {
        'learning_rate': lr,
        'batch_size': bs,
        'epochs': epochs,
        'dropout': dropout
    }
    torch.save(model.state_dict(), os.path.join(model_dir, 'best_model_bert.pt'))
    tqdm.write("Best model saved.")

with open(os.path.join(model_dir, 'grid_search_results_bert.json'), 'w') as f:
  json.dump(results, f, indent=2)

print(f"\nGrid search complete.")
print(f"Best Parameters: {best_params}")
print(f"Best F1 Score: {best_f1:.4f}")


Grid Search Progress:   0%|          | 0/36 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`



[1/36] Training with lr=1e-05, bs=8, epochs=3, dropout=0.1


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

Epoch:  33%|███▎      | 1/3 [02:28<04:57, 148.70s/it]

Epoch 1: Loss = 0.3654


...

Epoch 4: Loss = 0.4183



Epoch: 100%|██████████| 5/5 [10:03<00:00, 120.77s/it]

Epoch 5: Loss = 0.4005



Grid Search Progress: 100%|██████████| 36/36 [5:27:43<00:00, 546.22s/it]

Validation Accuracy: 0.6181, F1 Score: 0.5829

Grid search complete.
Best Parameters: {'learning_rate': 1e-05, 'batch_size': 16, 'epochs': 3, 'dropout': 0.1}
Best F1 Score: 0.7740


# Save JSON File containing best parameter setting.

In [ ]:
# Use independent code block to generate best parameter file in case of model training process being interrupted.
import json
import os

# path
model_dir = '/content/drive/My Drive/nlu-lab/bert_model'
grid_search_path = os.path.join(model_dir, 'grid_search_results_bert.json')
best_param_path = os.path.join(model_dir, 'best_params_bert.json')

# load best parameter
with open(grid_search_path, 'r') as f:
  results = json.load(f)

if results:
  best_result = max(results, key=lambda x: x['f1'])

  with open(best_param_path, 'w') as f:
    json.dump(best_result, f, indent=2)

  print(f"\nBest Parameters saved to: {best_param_path}")
  print(f"Best F1 Score: {best_result['f1']:.4f}")
  print(f"Params: {best_result}")
else:
  print("grid_search_results_bert.json is empty. Please check your training.")



Best Parameters saved to: /content/drive/My Drive/nlu-lab/bert_model/best_params_bert.json
Best F1 Score: 0.7740
Params: {'learning_rate': 1e-05, 'batch_size': 16, 'epochs': 3, 'dropout': 0.1, 'acc': 0.8633142085723928, 'f1': 0.7739955357142857}
